# How to build advanced Chatbot with session memory using LangChain
* Advanced Chatbot LLM App
    * Will be able to have conversation
    * Will remember previous interactions: will have memory
    * Will be able to have different memories for different user sessions
    * Will be able to remember limited number of messages: limited memory

## Concepts included
* Chat Model vs. LLM Model:
    *  Chat Model is based around messages
    *  LLM Model is based around raw text
* Chat History: allows Chat Model to remember previous interactions

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [4]:
!pip list | grep langchain

langchain                 0.2.1
langchain-community       0.2.1
langchain-core            0.2.2
langchain-openai          0.1.8
langchain-text-splitters  0.2.0

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip


## Connect with LLM and start conversation with it

In [6]:
from langchain_openai import ChatOpenAI

chatbot = ChatOpenAI(model="gpt-3.5-turbo")

In [7]:
from langchain_core.messages import HumanMessage

messagesToTheChatbot = [
    HumanMessage(content="My favourite color is blue."),
]

### Call ChatModel (LLM)

In [8]:
chatbot.invoke(messagesToTheChatbot)

AIMessage(content="Blue is such a calming and peaceful color. It's often associated with stability and trustworthiness. What do you like most about the color blue?", response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 13, 'total_tokens': 43}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-484b5288-6948-495f-88a1-cdd01e48f626-0', usage_metadata={'input_tokens': 13, 'output_tokens': 30, 'total_tokens': 43})

### Track operation in LangSmith
* [Open LangSmith here](https://smith.langchain.com/)

## Check if Chatbot remembers your favorite color

In [9]:
chatbot.invoke([
    HumanMessage(content="What is my favorite color?")
])

AIMessage(content="I'm sorry, I'm not able to know your favorite color as an AI assistant. Can you please tell me what your favorite color is?", response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 13, 'total_tokens': 42}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-75666902-eef4-4507-ba38-75b0b7e8a886-0', usage_metadata={'input_tokens': 13, 'output_tokens': 29, 'total_tokens': 42})

## Add memory to Chatbot
* We will use ChatMessageHistory package
* We will save Chatbot memory in python dictionary called chatbotMemory
* We will define get_session_history function to create session_id for each conversation

In [11]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

chatbotMemory = {}

# input: session_id, output: chatbotMemory[session_id]
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in chatbotMemory:
        chatbotMemory[session_id] = ChatMessageHistory()
    return chatbotMemory[session_id]


chatbot_with_message_history = RunnableWithMessageHistory(
    chatbot, 
    get_session_history
)

In [12]:
session1 = {"configurable": {"session_id": "001"}}

In [13]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="My favorite color is red.")],
    config=session1,
)

responseFromChatbot.content

"That's a bold and vibrant choice! Red is often associated with energy, passion, and strength. Do you have a specific reason why red is your favorite color?"

In [14]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="What's my favorite color?")],
    config=session1,
)

responseFromChatbot.content

'Your favorite color is red!'

## Change session and see what happens

In [15]:
session2 = {"configurable": {"session_id": "002"}}

In [16]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="What's my favorite color?")],
    config=session2,
)

responseFromChatbot.content

"I'm sorry, I don't have that information. Can you please tell me what your favorite color is?"

## Go back to session1 and see if memory is still there

In [17]:
session1 = {"configurable": {"session_id": "001"}}

In [18]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="What's my favorite color?")],
    config=session1,
)

responseFromChatbot.content

'Your favorite color is red!'

## ChatBot has session memory now. Check if it remembers conversation from session2

In [19]:
session2 = {"configurable": {"session_id": "002"}}

In [20]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="Mi name is Julio.")],
    config=session2,
)

responseFromChatbot.content

"Hello Julio! It's nice to meet you. If you'd like to share your favorite color with me, I'd be happy to remember it for future reference."

In [21]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=session2,
)

responseFromChatbot.content

'Your name is Julio.'

In [22]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="What is my favorite color?")],
    config=session1,
)

responseFromChatbot.content

'Your favorite color is red!'

## ChatBot now remembers each of conversations

## Importance to manage Conversation History

In [23]:
print(chatbotMemory)

{'001': InMemoryChatMessageHistory(messages=['Human: My favorite color is red.', AIMessage(content="That's a bold and vibrant choice! Red is often associated with energy, passion, and strength. Do you have a specific reason why red is your favorite color?", response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 13, 'total_tokens': 46}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e5fde172-58ef-4f73-b463-a2a463d0a1f7-0', usage_metadata={'input_tokens': 13, 'output_tokens': 33, 'total_tokens': 46}), AIMessage(content='Your favorite color is red!', response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 62, 'total_tokens': 68}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-99eebba4-d387-4a22-b717-1180116d043a-0', usage_metadata={'input_tokens': 62, 'output_tokens': 6, 'total_tokens': 68}), AIMessage(content='Your favor

In [24]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough


def limited_memory_of_messages(messages, number_of_messages_to_keep=2):
    return messages[-number_of_messages_to_keep:]

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

limitedMemoryChain = (
    RunnablePassthrough.assign(messages=lambda x: limited_memory_of_messages(x["messages"]))
    | prompt 
    | chatbot
)

In [25]:
chatbot_with_limited_message_history = RunnableWithMessageHistory(
    limitedMemoryChain,
    get_session_history,
    input_messages_key="messages",
)

### Add 2 more messages to session1 conversation

In [26]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="My favorite vehicles are Vespa scooters.")],
    config=session1,
)

responseFromChatbot.content

"That's a fun choice! Vespa scooters are known for their classic and stylish design. They are also great for getting around town and exploring new places. What is it about Vespa scooters that you like the most?"

In [27]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="My favorite city is San Francisco.")],
    config=session1,
)

responseFromChatbot.content

'San Francisco is a beautiful city with its iconic landmarks like the Golden Gate Bridge, Alcatraz Island, and cable cars. The diverse culture, amazing food scene, and stunning views of the bay make it a favorite for many people. What do you love most about San Francisco?'

## Chatbot memory has now 4 messages. Check Chatbot with limited memory

In [28]:
responseFromChatbot = chatbot_with_limited_message_history.invoke(
    {
        "messages": [HumanMessage(content="what is my favorite color?")],
    },
    config=session1,
)

responseFromChatbot.content

"I'm sorry, but as an AI assistant, I don't have access to personal information about you, such as your favorite color. If you'd like to share your favorite color with me, I'd be happy to discuss it further."

## Compare previous response with one provided by Chatbot with unlimited memory

In [30]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="what is my favorite color?")],
    config=session1,
)

responseFromChatbot.content

'Your favorite color is red!'